#  Script-ακι Network Check
Μικρό πρόγραμμα για έλεγχω δικτύου GPS

## Δομή Προγράμματος

*Όλο το πρόγραμμα (script) είναι χωρισμένο σε μικρότερες συναρτήσεις (functions) για καλύτερη κατανόηση και διαχείρηση.*

Τα functions, επιγραμματικά είναι:

```python
def main():
```
- Εδώ είναι μαζεμένες όλες οι λειτουργίες του προγράμματος και εκτελούνται όλες οι υπόλοιπες συναρτήσεις.  
<br>
<hr>

```python
def loadexcel(path: str) -> pd.DataFrame:
```

- Δέχεται για παράμετρο το **path** του **raw data** αρχείου ταχυμετρίας. Κάνει μια βασική προεπεξεργασία των δεδομένων και επιστρέφει έναν πίνακα του τύπου **DataFrame**  
- Ο πίνακας αυτός είναι ίδιος με τα **raw data** απλά δίχως κάποιες στήλες  
<br>
<hr>

```python
def filterData(df: pd.DataFrame) -> list:
```

- Δέχεται για παράμετρο ένα **DataFrame** (προφανός το αποτέλεσμα του `loadexcel`) και κάνει ένα "φιλτράρισμα". 
- Επιστρέφει μια λίστα απο **Dictionaries**  
<br>
<hr>

```python
def createRawDf(filteredData: list) -> pd.DataFrame:
```

- Δέχεται μια λίστα και επιστρέφει ενα **DataFrame** 
- Είναι παρόμοιο με το `filterData` απλά αντί για πολλά **Dictionaries** έχουμε ένα **DataFrame** το οποίο είναι καλύτερα οργανομένο και είναι πιο εύκολη η πρόσβαση στα δεδομένα της.  
<br>
<hr>

```python
def getCoords(RAWDataDf: pd.DataFrame) -> pd.DataFrame:
```

- Δέχετε ένα **DataFrame** με βάση το οποίο δημιουργεί και επιστρέφει τρείς διαφορετικούς πίνακες τύπου **DataFrame**.
- Ο ποιό σημαντικός απο αυτούς περιλαμβάνει τα τριγωνομετρικά σημεία που χρησιμοποιήθηκαν στο **raw data** με βἀση τα οποία θα γίνουν οι όποιες συγκρίσεις και υπολογισμοί.
<br>
<hr>

```python
def calculate(RAWDataDf: pd.DataFrame, distanceList: list, heightDiffList: list) -> pd.DataFrame:
```

- Όπως λέει και το όνομα, κάνει τους υπολογισμούς. Σε αυτή την φάση οι υπολογσιμοί δεν είναι της μορφής της πράξης κελιών.
<br>
<hr>

```python
def createCalculationsExcel(RAWDataDf: pd.DataFrame, calcValuesDf: pd.DataFrame, differencesDf: pd.DataFrame, DataCoordsDf: pd.DataFrame) -> str:
```

-Δημιουργεί ένα εξέλ όπου αποθηκεύονται όλοι οι πίνακες και υπολογισμοί σε καρτέλες. 
<br>
<hr>

```python
def createFinalFormatedExcel(RAWDataDf: pd.DataFrame, DataCoordsDf: pd.DataFrame, path: str):
```

- Έδω γίνονται οι πράξεις στα κελιά και όχι στους πίνακες με τους οποίους δουλεύαμε μέχρι τώρα.
- Επιπλέον όλες οι μορφοποιήσεις πραγματοποιούνται εδώ

## Φόρτωση βιβλιοθηκών 

`pandas` για επεξεργασία δεδομένων  
`numpy` για έτσι  
`math` για πράξεις  
`os` για διαχείρηση/εκτέλεση εργασιών συστήματος  
`openpyxl` για διαμόρφωση και επεξεργασία αρχείων excel (.xlsx)  

Από το τελευταίο δεν φορτώνω όλη την βιβλιοθήκη αλλά μόνο κάποιες συγκεκριμένες κλάσεις και συναρτήσεις όπως:  
> - `Workbook` και `load_workbook` για δημιουργία και άνοιγμα αρχείων  
> - `Border`, `Side`, `Alignment` και `Font` για επεξεργασία/format κελιών  
> - `get_column_letter` το οποίο είναι συνάρτηση για ανάκτηση γράμματος στήλης ενός κελιού 

In [1]:
import pandas as pd
import numpy as np
import math
import os

from openpyxl import Workbook
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font 
from openpyxl.utils import get_column_letter

In [3]:

def main():
    rawUnfilteredDataPath = input("path to file: ")
    mainDf = loadExcel(rawUnfilteredDataPath)
    filteredData = filterData(mainDf)
    RAWDataDf  = createRawDf(filteredData)
    CoordsDf, CalculationsDf, DifferencesDf = getCoords(RAWDataDf)
    filePath = createCalculationsExcel(RAWDataDf, CalculationsDf, DifferencesDf, CoordsDf)
    createFinalFormatedExcel(RAWDataDf, CoordsDf, filePath)


In [4]:
def loadExcel(path: str) -> pd.DataFrame:
    with os.scandir(path) as entries:
        dirL = [entry.name for entry in entries if entry.name == "raw.xlsx"]
    
    try:
        fullPath = str(path + "\\" + dirL[0])
    except Exception:
        print("Error while trying to get the full .xlsx file path.")


    mainDf = pd.read_excel(io=fullPath, header=None)
    mainDf.drop([1, 2, 3, 4, 5, 6, 7],
                axis=1,
                inplace=True)  # Drop useless columns

    mainDf.rename(
        columns={
            0: "StationName",
            8: "Horizontal Distance",
            9: "Height Difference",
            10: "Corrected H Dist",
            11: "Corrected H Diff"
        }, inplace=True)
    
    return mainDf